# Setup PySpark

In [56]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
import pyspark.sql.functions as F
import os

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#os.environ['PYSPARK_SUBMIT_ARGS'] = \
#  '--org.postgresql.Driver /Users/tbiytc/Desktop/LifeZ_Data/postgresql-42.4.1.jar pyspark-shell'
#sc = SparkContext()
conf = SparkConf()  # create the configuration
conf.setMaster("local")
conf.set("spark.jars", "./postgresql-42.4.1.jar")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark = SparkSession.builder\
                    .config(conf = conf)\
                    .appName('test').getOrCreate()
sc = SparkContext.getOrCreate()

In [2]:
spark

## Demo

In [3]:
data = [("English", "1000000"), ("Chinese", "10000000000")]
columns = ["language", "users_count"]
df = spark.createDataFrame(data).toDF(*columns)
df.show()

+--------+-----------+
|language|users_count|
+--------+-----------+
| English|    1000000|
| Chinese|10000000000|
+--------+-----------+



In [6]:
df.write.format('jdbc').options(
  url='jdbc:postgresql://localhost:5432/test',
  driver='org.postgresql.Driver',
  dbtable='test_table',
  user='username',
  password='password').mode('append').save()

---

# 10000-Patients Data

In [10]:
# Tables
spark.read.format('jdbc'). \
     options(
         url='jdbc:postgresql://localhost:5432/test', # database url (local, remote)
         dbtable='information_schema.tables',
         user='username',
         password='password',
         driver='org.postgresql.Driver'). \
     load().show()

+-------------+------------+--------------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|          table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+--------------------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|       tbiytc|      public|          test_table|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|       tbiytc|  pg_catalog|        pg_statistic|BASE TABLE|                        null|   

In [80]:
import pandas as pd
import os

def read_patient_files(filename):
    with open(filename, 'r') as f:
        file_content = f.readlines()
    return file_content

def write_to_db(data_frame, table_name):
    data_frame.write.format('jdbc').options(
              url='jdbc:postgresql://localhost:5432/tbiytc',
              driver='org.postgresql.Driver',
              dbtable=table_name,
              user='tbiytc',
              password='jason1995').mode('append').save()
    out_message = f"Data frame {data_frame} has been appended to table {table_name} in the PostgreSQL database."
    return out_message

In [18]:
data_dir = "./100000-Patients/"

Read data

In [74]:
admission = spark.read.csv(data_dir+"AdmissionsCorePopulatedTable.txt", sep="\t", header=True)
patient = spark.read.csv(data_dir+"PatientCorePopulatedTable.txt", sep="\t", header=True)
diagnosis = spark.read.csv(data_dir+"AdmissionsDiagnosesCorePopulatedTable.txt", sep="\t", header=True)
lab = spark.read.csv(data_dir+"LabsCorePopulatedTable.txt", sep=r'\t', header=True)

Design database in Beekeeper Studio - DONE

Column Mapping

In [62]:
admission_col_map = {"PatientID": "patient_id", "AdmissionID": "admission_id", 
                     "AdmissionStartDate": "admission_start_date", "AdmissionEndDate": "admission_end_date"}
admission_val_map = {"PatientID": "STRING", "AdmissionID": "STRING",
                     "AdmissionStartDate": "DATE", "AdmissionEndDate": "DATE"}

In [66]:
patient_col_map = {"PatientID": "patient_id", "PatientGender": "gender", 
                   "PatientDateOfBirth": "date_of_birth", "PatientRace": "ethnicity",
                   "PatientMaritalStatus": "marital_status", "PatientLanguage": "language",
                   "PatientPopulationPercentageBelowPoverty": "population_pct_below_poverty"}
patient_val_map = {"PatientID": "STRING", "PatientGender": "STRING",
                   "PatientDateOfBirth": "DATE", "PatientRace": "STRING",
                   "PatientMaritalStatus": "STRING", "PatientLanguage": "STRING",
                   "PatientPopulationPercentageBelowPoverty": "NUMERIC"}

In [70]:
diagnosis_col_map = {"PatientID": "patient_id", "AdmissionID": "admission_id", 
                     "PrimaryDiagnosisCode": "primary_diagnosis_code", "PrimaryDiagnosisDescription": "primary_diagnosis_description"}
diagnosis_val_map = {"PatientID": "STRING", "AdmissionID": "STRING",
                     "PrimaryDiagnosisCode": "STRING", "PrimaryDiagnosisDescription": "STRING"}

In [90]:
lab_col_map = {"PatientID": "patient_id", "AdmissionID": "admission_id", 
               "LabName": "lab_name", "LabValue": "lab_value",
               "LabUnits": "lab_units", "LabDateTime": "lab_datetime"}
lab_val_map = {"PatientID": "STRING", "AdmissionID": "STRING", 
               "LabName": "STRING", "LabValue": "NUMERIC",
               "LabUnits": "STRING", "LabDateTime": "DATE"}

In [77]:
df = spark.read.format('jdbc').\
                options(
    url='jdbc:postgresql://localhost:5432/test', 
    dbtable='lab',
    user='username',
    password='password',
    driver='org.postgresql.Driver').load()
df.printSchema()
lab.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- admission_id: string (nullable = true)
 |-- lab_name: string (nullable = true)
 |-- lab_value: decimal(38,18) (nullable = true)
 |-- lab_units: string (nullable = true)
 |-- lab_datetime: date (nullable = true)

root
 |-- PatientID: string (nullable = true)
 |-- AdmissionID: string (nullable = true)
 |-- LabName: string (nullable = true)
 |-- LabValue: string (nullable = true)
 |-- LabUnits: string (nullable = true)
 |-- LabDateTime: string (nullable = true)



Necessary Transformations

In [60]:
admission_transformed = \
        admission.select(F.col("PatientID").cast(admission_val_map["PatientID"]).alias(admission_col_map["PatientID"]),
                         F.col("AdmissionID").cast(admission_val_map["AdmissionID"]).alias(admission_col_map["AdmissionID"]),
                         F.col("AdmissionStartDate").cast(admission_val_map["AdmissionStartDate"]).alias(admission_col_map["AdmissionStartDate"]),
                         F.col("AdmissionEndDate").cast(admission_val_map["AdmissionEndDate"]).alias(admission_col_map["AdmissionEndDate"]))

In [67]:
patient_transformed = \
patient.select(F.col("PatientID").cast(patient_val_map["PatientID"]).alias(patient_col_map["PatientID"]),
               F.col("PatientGender").cast(patient_val_map["PatientGender"]).alias(patient_col_map["PatientGender"]),
               F.col("PatientDateOfBirth").cast(patient_val_map["PatientDateOfBirth"]).alias(patient_col_map["PatientDateOfBirth"]),
               F.col("PatientRace").cast(patient_val_map["PatientRace"]).alias(patient_col_map["PatientRace"]),
               F.col("PatientMaritalStatus").cast(patient_val_map["PatientMaritalStatus"]).alias(patient_col_map["PatientMaritalStatus"]),
               F.col("PatientLanguage").cast(patient_val_map["PatientLanguage"]).alias(patient_col_map["PatientLanguage"]),
               F.col("PatientPopulationPercentageBelowPoverty")\
                    .cast(patient_val_map["PatientPopulationPercentageBelowPoverty"]).alias(patient_col_map["PatientPopulationPercentageBelowPoverty"]))

In [71]:
diagnosis_transformed = \
        diagnosis.select(F.col("PatientID").cast(diagnosis_val_map["PatientID"]).alias(diagnosis_col_map["PatientID"]),
                         F.col("AdmissionID").cast(diagnosis_val_map["AdmissionID"]).alias(diagnosis_col_map["AdmissionID"]),
                         F.col("PrimaryDiagnosisCode").cast(diagnosis_val_map["PrimaryDiagnosisCode"]).alias(diagnosis_col_map["PrimaryDiagnosisCode"]),
                         F.col("PrimaryDiagnosisDescription")\
                              .cast(diagnosis_val_map["PrimaryDiagnosisDescription"]).alias(diagnosis_col_map["PrimaryDiagnosisDescription"]))

In [91]:
lab_transformed = \
        lab.select(F.col("PatientID").cast(lab_val_map["PatientID"]).alias(lab_col_map["PatientID"]),
                   F.col("AdmissionID").cast(lab_val_map["AdmissionID"]).alias(lab_col_map["AdmissionID"]),
                   F.col("LabName").cast(lab_val_map["LabName"]).alias(lab_col_map["LabName"]),
                   F.col("LabValue").cast(lab_val_map["LabValue"]).alias(lab_col_map["LabValue"]),
                   F.col("LabUnits").cast(lab_val_map["LabUnits"]).alias(lab_col_map["LabUnits"]),
                   F.col("LabDateTime").cast(lab_val_map["LabDateTime"]).alias(lab_col_map["LabDateTime"]))

Fit table into database

In [83]:
write_to_db(admission_transformed, "admission")

'Data frame DataFrame[patient_id: string, admission_id: string, admission_start_date: date, admission_end_date: date] has been appended to table admission in the PostgreSQL database.'

In [84]:
write_to_db(patient_transformed, "patient")

'Data frame DataFrame[patient_id: string, gender: string, date_of_birth: date, ethnicity: string, marital_status: string, language: string, population_pct_below_poverty: decimal(10,0)] has been appended to table patient in the PostgreSQL database.'

In [85]:
write_to_db(diagnosis_transformed, "diagnosis")

'Data frame DataFrame[patient_id: string, admission_id: string, primary_diagnosis_code: string, primary_diagnosis_description: string] has been appended to table diagnosis in the PostgreSQL database.'

In [92]:
write_to_db(lab_transformed, "lab")

'Data frame DataFrame[patient_id: string, admission_id: string, lab_name: string, lab_value: decimal(10,0), lab_units: string, lab_datetime: date] has been appended to table lab in the PostgreSQL database.'

Test out the ability to query the database

In [93]:
# Tables
df_test = spark.read.format('jdbc'). \
                     options(
                         url='jdbc:postgresql://localhost:5432/test', # database url (local, remote)
                         dbtable='admission',
                         user='username',
                         password='password',
                         driver='org.postgresql.Driver').load()